In [3]:
import pandas as pd
from urllib.request import urlopen
import xml.etree.ElementTree as ET  

In [5]:
urls_data = pd.read_csv('./law_links.csv')

In [ ]:
link_front = "https://www.law.go.kr"

decrees_list = []
for index_num in range(0,len(urls_data['법령상세링크'])):
    url = link_front+urls_data['법령상세링크'][index_num]
    #get xml
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    decrees_count = len(xtree.find('조문'))
    #조문번호, 조문내용 가져오기
    for j in range(0, decrees_count):
        decrees_info_list =[]
        decress_tree =  xtree.find('조문')[j]
        decress_tree_str = ET.tostring(decress_tree, encoding='unicode') 
        if "조(" not in decress_tree_str:
            continue                
        else:
            decrees_num = xtree.find('조문')[j].find('조문번호').text
            decrees_contents= xtree.find('조문')[j].find('조문내용').text
            
            #항, 호 추출
            for object in decress_tree.iter("항"):
                for i in range(0, len(object)):                   
                    xml_str = ET.tostring(object[i], encoding='unicode')                    
                    if "<항내용>" in xml_str:
                        hang_content = object.find("항내용").text
                        decrees_contents = decrees_contents+ hang_content
                    elif "<호>" in xml_str:
                        #호를 인식해도 해당 내용을 못가져오는 경우가 있음.. 
                        # print(object.find("호").find("호내용").text)
                        xml_str_split = xml_str.split("<호내용>")
                        xml_str_replace1=xml_str_split[1].replace("</호>","").strip()
                        xml_str_replace2=xml_str_replace1.replace("</호내용>","")
                        if "<목>" in xml_str_replace2:
                            m_split = xml_str_replace2.split("<목>")
                            xml_str_replace2 = m_split[0].strip()
                            for k in range(0, len(m_split)):                    
                                if "<목내용>" in m_split[k]:
                                    mc_split = m_split[k].split("<목내용>")
                                    mc_str= mc_split[1].replace("</목내용>","").strip()
                                    mc_str = mc_str.replace("</목>","")
                                    #print(mc_str)
                                    xml_str_replace2 = xml_str_replace2+" "+ mc_str
                        
                        if ("&lt;" in xml_str_replace2) or ("&gt;" in xml_str_replace2):
                            xml_str_replace2=xml_str_replace2.replace("&lt;","<")
                            xml_str_replace2=xml_str_replace2.replace("&gt;",">")
                        decrees_contents = decrees_contents+xml_str_replace2                          
            
            decrees_info_list.append(urls_data['법령명'][index_num])
            decrees_info_list.append(urls_data['법령MST'][index_num])
            decrees_info_list.append(urls_data['법령ID'][index_num])
            decrees_info_list.append(urls_data['시행일자'][index_num])
            decrees_info_list.append(urls_data['공포번호'][index_num])
            decrees_info_list.append(urls_data['법령구분명'][index_num])
            decrees_info_list.append(decrees_num)
            decrees_info_list.append(decrees_contents)
            decrees_list.append(decrees_info_list)
        
        


In [ ]:
columns = ['법령명', '법령MST', '법령ID', '시행일자', '공포번호', '법령구분명', '조문번호', '조문내용']
law_df = pd.DataFrame(decrees_list, columns = columns)
law_df.head()

In [ ]:
#csv로 저장
law_df.to_csv('./law_data.csv', index=False, encoding='utf-8_*_' )